<a href="https://colab.research.google.com/github/Rudy-Nzau/EDA_Feat-Eng_OHE_Standardisation_PCA/blob/main/EDA_Feat_Eng_OHE_Standardisation_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=blue>**One Hot Encoding, Standardization, PCA: Data preparation for segmentation in python**

##**1.The Data**

For this exercise, we will be working with clickstream data from an online store offering clothing for pregnant women. It has data from April 2008 to August 2008 and includes variables like product category, location of the photo on the webpage, country of origin of the IP address and product price in US dollars. The reason I chose this dataset is that clickstream data is becoming a very important source of providing fine-grained information about customer behaviour. It also provides us a dataset with typical challenges like high dimensionality, need for feature engineering, presence of categorical variables and different scales of fields.

##**2.Exploratory Data Analysis (EDA)**

In [7]:
# Read dataset and look at top records
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/e-shop clothing 2008.csv', delimiter=";")
df.tail(10)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
165464,2008,8,13,3,29,24023,1,A15,14,5,2,33,2,1
165465,2008,8,13,4,29,24023,1,A16,1,6,1,33,2,1
165466,2008,8,13,5,29,24023,4,P11,4,4,2,38,1,1
165467,2008,8,13,6,29,24023,4,P18,2,6,1,28,2,1
165468,2008,8,13,7,29,24023,4,P13,4,5,1,38,1,1
165469,2008,8,13,1,29,24024,2,B10,2,4,1,67,1,1
165470,2008,8,13,1,9,24025,1,A11,3,4,1,62,1,1
165471,2008,8,13,1,34,24026,1,A2,3,1,1,43,2,1
165472,2008,8,13,2,34,24026,3,C2,12,1,1,43,1,1
165473,2008,8,13,3,34,24026,2,B2,3,1,2,57,1,1


In [3]:
#Check the number of rows and columns and their types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  int64 
 1   month                    165474 non-null  int64 
 2   day                      165474 non-null  int64 
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  int64 
 5   session ID               165474 non-null  int64 
 6   page 1 (main category)   165474 non-null  int64 
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  int64 
 9   location                 165474 non-null  int64 
 10  model photography        165474 non-null  int64 
 11  price                    165474 non-null  int64 
 12  price 2                  165474 non-null  int64 
 13  page                     165474 non-null  int64 
dtypes: int64(13), object

In [4]:
# Convert categorical variables to string
cat_vars = ['year', 'month', 'day', 'country', 'session ID',
               'page 1 (main category)', 'page 2 (clothing model)',   'colour',
               'location', 'model photography', 'price 2', 'page']
df[cat_vars] = df[cat_vars].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  object
 1   month                    165474 non-null  object
 2   day                      165474 non-null  object
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  object
 5   session ID               165474 non-null  object
 6   page 1 (main category)   165474 non-null  object
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  object
 9   location                 165474 non-null  object
 10  model photography        165474 non-null  object
 11  price                    165474 non-null  int64 
 12  price 2                  165474 non-null  object
 13  page                     165474 non-null  object
dtypes: int64(2), object(

In [5]:
# Check properties of numeric fields
df.describe()

,order,price
count,165474.000000,165474.000000
mean,9.817476,43.802507
std,13.478411,12.548131
min,1.000000,18.000000
25%,2.000000,33.000000
50%,6.000000,43.000000
75%,12.000000,52.000000
max,195.000000,82.000000


As seen in figure 4, the product price (field name: ‘price’) is on a much larger scale than sequence of clicks during one session (field name: ‘order’). This means that we will have to standardize these fields to bring them to the same scale as distance based models like K-means are affected by the scale of the fields.

##**3.Feature Engineering**

We create the following features while aggregating at the product level:

1. Most frequently occurring product colour, day of browsing, country, photo type (profile, en face), price type (higher or lower than category average), page number within website and location of the product’s photo on the page (using the mode function)

2. Total number of unique session IDs (using the nununique function)

3. Median, minimum and maximum of sequence of clicks during one session and product price (using the median, min and max function)


In [8]:
# Feature Engineering
from scipy.stats import mode 
df2 = df.groupby(['country','page 1 (main category)',
                  'page 2 (clothing model)']).agg(  median_no_of_clicks_per_session=('order', 'median'),
                                                    min_no_of_clicks_per_session=('order', 'max'),
                                                    max_no_of_clicks_per_session=('order', 'min'),
                                                    median_price=('price', 'median'),
                                                    min_price=('price', 'max'),
                                                    max_price=('price', 'min'),
                                                    total_number_of_sessions =('session ID', pd.Series.nunique),
                                                    most_frequent_day=('day', lambda x: mode(x)[0][0]),
                                                    most_frequent_colour=('colour', lambda x: mode(x)[0][0]),
                                                    most_frequent_location=('location', lambda x: mode(x)[0][0]),
                                                    most_frequent_photo_type=('model photography', lambda x: mode(x)[0][0]),
                                                    most_frequent_price_type =('price 2', lambda x: mode(x)[0][0]),
                                                    most_frequent_page_number =('page', lambda x: mode(x)[0][0])
                                                                            )
df2

<ipython-input-8-4c295cf2aef6>:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_frequent_day=('day', lambda x: mode(x)[0][0]),
<ipython-input-8-4c295cf2aef6>:12: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_frequent_colour=('colour', lambda x: mode(x)[

median_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                    
1       3                      C30                                                  2.0   
        4                      P17                                                  2.0   
2       1                      A1                                                   1.0   
                               A10                                                  2.0   
                               A11                                                  2.0   
...                                                                                 ...   
47      1                      A6                                                   2.0   
        2                      B24                                                  1.0   
        3                      C41                                                  3.0   
                               C5                                                   2.0   
                               C9                                                   1.0   

                                                        min_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                                 2   
        4                      P17                                                 3   
2       1                      A1                                                  1   
                               A10                                                 2   
                               A11                                                 2   
...                                                                              ...   
47      1                      A6                                                  2   
        2                      B24                                                 1   
        3                      C41                                                 3   
                               C5                                                  2   
                               C9                                                  1   

                                                        max_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                                 2   
        4                      P17                                                 1   
2       1                      A1                                                  1   
                               A10                                                 2   
                               A11                                                 2   
...                                                                              ...   
47      1                      A6                                                  2   
        2                      B24                                                 1   
        3                      C41                                                 3   
                               C5                                                  2   
                               C9                                                  1   

                                                        median_price  \
country page 1 (main category) page 2 (clothing model)                 
1       3                      C30                              28.0   
        4                      P17                              38.0   
2       1                      A1                               28.0   
                               A10                              38.0   
                               A11                              62.0   
...                                                              ...   
47    

##**4.One Hot Encoding**

In [9]:
# One hot encoding - to convert categorical data to continuous
cat_vars = ['most_frequent_day',
           'most_frequent_colour', 'most_frequent_location',
           'most_frequent_photo_type', 'most_frequent_price_type',
           'most_frequent_page_number']
df2[cat_vars] = df2[cat_vars].astype(str)
df3 = pd.get_dummies(df2)
df3.head(5)

median_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                    
1       3                      C30                                                  2.0   
        4                      P17                                                  2.0   
2       1                      A1                                                   1.0   
                               A10                                                  2.0   
                               A11                                                  2.0   

                                                        min_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                                 2   
        4                      P17                                                 3   
2       1                      A1                                                  1   
                               A10                                                 2   
                               A11                                                 2   

                                                        max_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                                 2   
        4                      P17                                                 1   
2       1                      A1                                                  1   
                               A10                                                 2   
                               A11                                                 2   

                                                        median_price  \
country page 1 (main category) page 2 (clothing model)                 
1       3                      C30                              28.0   
        4                      P17                              38.0   
2       1                      A1                               28.0   
                               A10                              38.0   
                               A11                              62.0   

                                                        min_price  max_price  \
country page 1 (main category) page 2 (clothing model)                         
1       3                      C30                             28         28   
        4                      P17                             38         38   
2       1                      A1                              28         28   
                               A10                             38         38   
                               A11                             62         62   

                                                        total_number_of_sessions  \
country page 1 (main category) page 2 (clothing model)                             
1       3                      C30                                             1   
        4                      P17                                             1   
2       1                      A1                                              1   
                               A10                                             1   
                               A11                                             1   

                                                        most_frequent_day_1  \
country page 1 (main category) page 2 (clothing model)                        
1       3                      C30                                        0   
        4                      P17                                        0   
2       1                      A1                                         0   
                               A10                                        0   
                               A11                     

##**5.Standardization**

In [10]:
# Standardizing
from sklearn.preprocessing import StandardScaler
con_vars = ['median_no_of_clicks_per_session', 'min_no_of_clicks_per_session',
           'max_no_of_clicks_per_session', 'median_price', 'min_price',
           'max_price', 'total_number_of_sessions']
scaler = StandardScaler()
df3[con_vars]=scaler.fit_transform(df3[con_vars])
df3.head(5)

median_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                    
1       3                      C30                                            -0.784923   
        4                      P17                                            -0.784923   
2       1                      A1                                             -0.883016   
                               A10                                            -0.784923   
                               A11                                            -0.784923   

                                                        min_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                         -0.740009   
        4                      P17                                         -0.715043   
2       1                      A1                                          -0.764976   
                               A10                                         -0.740009   
                               A11                                         -0.740009   

                                                        max_no_of_clicks_per_session  \
country page 1 (main category) page 2 (clothing model)                                 
1       3                      C30                                         -0.445811   
        4                      P17                                         -0.550030   
2       1                      A1                                          -0.550030   
                               A10                                         -0.445811   
                               A11                                         -0.445811   

                                                        median_price  \
country page 1 (main category) page 2 (clothing model)                 
1       3                      C30                         -1.197785   
        4                      P17                         -0.367561   
2       1                      A1                          -1.197785   
                               A10                         -0.367561   
                               A11                          1.624976   

                                                        min_price  max_price  \
country page 1 (main category) page 2 (clothing model)                         
1       3                      C30                      -1.197785  -1.197785   
        4                      P17                      -0.367561  -0.367561   
2       1                      A1                       -1.197785  -1.197785   
                               A10                      -0.367561  -0.367561   
                               A11                       1.624976   1.624976   

                                                        total_number_of_sessions  \
country page 1 (main category) page 2 (clothing model)                             
1       3                      C30                                     -0.254235   
        4                      P17                                     -0.254235   
2       1                      A1                                      -0.254235   
                               A10                                     -0.254235   
                               A11                                     -0.254235   

                                                        most_frequent_day_1  \
country page 1 (main category) page 2 (clothing model)                        
1       3                      C30                                        0   
        4                      P17                                        0   
2       1                      A1                                         0   
                               A10                                        0   
                               A11                     

##**6.PCA**

In [11]:
# PCA
from sklearn.decomposition import PCA
# Loop Function to identify number of principal components that explain at least 85% of the variance
for comp in range(3, df3.shape[1]):
    pca = PCA(n_components= comp, random_state=42)
    pca.fit(df3)
    comp_check = pca.explained_variance_ratio_
    final_comp = comp
    if comp_check.sum() > 0.85:
        break
        
Final_PCA = PCA(n_components= final_comp,random_state=42)
Final_PCA.fit(df3)
cluster_df=Final_PCA.transform(df3)
num_comps = comp_check.shape[0]
print("Using {} components, we can explain {}% of the variability in the original data.".format(final_comp,comp_check.sum()))

Using 15 components, we can explain 0.8522617304004931% of the variability in the original data.


As seen in figure 8, 15 components are able to explain 85% of the variance in our dataset. We can now use these features in our unsupervised models like K means, DBSCAN, hierarchical clustering etc to segment our products.

## **Conclusion**

In this post, we learnt about the steps needed to prepare data for segmentation analysis.

Specifically, we learned:

* How we should perform exploratory data analysis by looking at the 
data, the field types and the properties of numeric fields.

* Examples of what kind of features we can create from the raw categorical and continuous fields.

* How to implement one hot encoding in python as well as ordinal encoding

* Various types of scaling techniques and how to choose between them
What is PCA and how to use it in python for feature reduction